In [15]:
!pip install -q chromadb
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q transformers
!pip install -q langchain
!pip install -q langchain-community==0.2.1 langchain-core==0.2.1
!pip install -q -U bitsandbytes
!pip install -q sentence_transformers
!pip install -q -U accelerate
!pip install -q unstructured
!pip install -q langchain-huggingface

In [2]:
!transformers-cli env


Copy-and-paste the text below in your GitHub issue and FILL OUT the two last points.

- `transformers` version: 4.41.2
- Platform: Windows-10-10.0.22621-SP0
- Python version: 3.11.9
- Huggingface_hub version: 0.23.3
- Safetensors version: 0.4.3
- Accelerate version: 0.30.1
- Accelerate config: 	not found
- PyTorch version (GPU?): 2.3.1+cu121 (True)
- Tensorflow version (GPU?): not installed (NA)
- Flax version (CPU?/GPU?/TPU?): not installed (NA)
- Jax version: not installed
- JaxLib version: not installed
- Using GPU in script?: <fill in>
- Using distributed or parallel set-up in script?: <fill in>



In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)

C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#################################################################
# Tokenizer
#################################################################

model_name='meta-llama/Meta-Llama-3-8B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "bfloat16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Device partition
device_map = "auto"

In [4]:
#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [5]:
#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    low_cpu_mem_usage = True,
    device_map = "auto"
)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.70s/it]


## LangChain

In [6]:
CHROMA_PATH = "chroma"
DATA_PATH = "data"

In [7]:
from bs4 import BeautifulSoup
import os

def XMLLoader(folder_path):
    reportData_list = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                content = file.read()

            soup = BeautifulSoup(content, 'xml')

            reportData = {}
            for tag in soup.find_all():
                if tag.name != "report":
                    reportData[tag.name] = tag.text.strip()

            reportData["filename"] = filename
            reportData_list.append(reportData)

    return reportData_list

In [8]:
data =  XMLLoader("data")

In [9]:
from langchain.docstore.document import Document

documents = []
for doc in data:

    document = Document(page_content=doc["report_text"], metadata={"source": doc["filename"],
                                                                   "subtype": doc["subtype"],
                                                                   "type": doc["type"],
                                                                   "chief_complaint": doc["chief_complaint"],
                                                                   "admit_diagnosis": doc["admit_diagnosis"],
                                                                   "discharge_diagnosis": doc["discharge_diagnosis"],
                                                                   "year": doc["year"],
                                                                   "downlaod_time": doc.get("download_time", ""), # Algunos no tiemem este campo
                                                                   "deid": doc["deid"]})
    documents.append(document)

In [10]:
#documents[1].metadata["subtype"]="TEST"
#print(documents[7])

In [11]:
from langchain.embeddings import SentenceTransformerEmbeddings

embedding_model = "BAAI/bge-small-en-v1.5"

# https://huggingface.co/sentence-transformers
# https://huggingface.co/spaces/mteb/leaderboard
embeddings = SentenceTransformerEmbeddings(model_name=embedding_model)

C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 500

# LLama3 - Ventana de contexto de 8000 tokens
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer,
    chunk_size=chunk_size,
    chunk_overlap=100,
    add_start_index=True
)

chunks = text_splitter.split_documents(documents)
print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

Split 41 documents into 198 chunks.


In [13]:
from langchain.vectorstores.chroma import Chroma

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)

C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [14]:
from langchain_huggingface.llms import HuggingFacePipeline

# Pipeline for inference
def load_model():
    text_generation_pipeline = pipeline(
        model=model, # Model loaded in the first part
        tokenizer=tokenizer, # Tokenizer loaded in the first part
        task="text-generation",
        temperature=0.1,
        repetition_penalty=1.1,
        return_full_text=True,
        max_new_tokens=500,
        do_sample=True
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline, model_id="meta-llama/Meta-Llama-3-8B-Instruct")
    return hf

In [15]:
llm = load_model()

### MultiQueryRetriever

Mediante un LLM se generan múltiples queries (prompts) desde distintas perspectivas para un prompt introducido por el user. Por cada query hace un retrieval de los documentos reelevantes para obtener un set mas grande de posibles documentos con mayor variedad.

In [16]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever = vectorstore.as_retriever(), 
    llm=llm,
)

In [17]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [18]:
# Default retriever (No custom prompt template)

#uestion = "Information about the patient John Smith"
#out = multi_query_retriever.invoke(question)

In [19]:
#len(out)

### Custom prompt template

In [20]:
from langchain_core.prompts import PromptTemplate

template = """
    <|start_header_id|>system<|end_header_id|> 
    You are an AI language model assistant. Your task is to generate three (3) different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions separated by newlines.
    <|eot_id|>
    <|start_header_id|>user<|end_header_id|> Original question: {question}<|eot_id|>"""

In [21]:
PROMPT = PromptTemplate(
    template=template, input_variables=["question"]
)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm, prompt=PROMPT
)

In [22]:
#docs = retriever_from_llm.get_relevant_documents("Information about John Smith")
#len(docs)

## Full Chain

In [23]:
from langchain.prompts import PromptTemplate

prompt_template = """
    <|start_header_id|>system<|end_header_id|> Answer as an expert doctor. Analyse the context prvided below to answer the query. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context:
    {context}
    <|eot_id|>
    <|start_header_id|>user<|end_header_id|> This is the Query: {question}<|eot_id|>"""

promptTemplate = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)

In [24]:
from langchain.chains import LLMChain

llm_chain = LLMChain(llm=llm, prompt=promptTemplate)

C:\Users\201902452\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [25]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
 {"context": retriever_from_llm, "question": RunnablePassthrough()}
    | llm_chain
)

In [37]:
answer = rag_chain.invoke("What medications were administrated to patients with  chronic obstructive pulmonary disease?")
print(answer['text'])
len(answer['context'])

INFO:langchain.retrievers.multi_query:Generated queries: ['<|start_header_id|>system<|end_header_id|> ', '    You are an AI language model assistant. Your task is to generate three (3) different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.', '    Provide these alternative questions separated by newlines.', '    <|eot_id|>', '    <|start_header_id|>user<|end_header_id|> Original question: What medications were administrated to patients with  chronic obstructive pulmonary disease?<|eot_id|>assistant', '', 'Here are three alternative versions of the original question:', '', 'What medications have been prescribed for individuals diagnosed with chronic obstructive pulmonary disease (COPD)?', '', 'Which pharmaceutical treatments have been used to manage symptoms in patients with chronic 


    <|start_header_id|>system<|end_header_id|> Answer as an expert doctor. Analyse the context prvided below to answer the query. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context:
    [Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '786.05', 'chief_complaint': 'PLEURAL EFFUSION', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '428.0,427.31,242.90,V10.3,250.00,401.9,', 'downlaod_time': '', 'source': 'report36.xml', 'start_index': 0, 'subtype': 'CONSULT', 'type': 'HP', 'year': '2007'}), Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '733.82', 'chief_complaint': 'PINS STICKING THROUGH FEMUR', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '733.82,996.49,V45.4,305.1,305.20,', 'downlaod_time': '', 'source': 'report20.xml', 'start_index': 0, 'subtype': 'GIM CONSULT', 'type': 'HP', 'year

20

In [38]:
answer = rag_chain.invoke("What treatment or management plan is proposed for a patient's closed head injury and multiple contusions?")
print(answer['text'])

INFO:langchain.retrievers.multi_query:Generated queries: ['<|start_header_id|>system<|end_header_id|> ', '    You are an AI language model assistant. Your task is to generate three (3) different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.', '    Provide these alternative questions separated by newlines.', '    <|eot_id|>', "    <|start_header_id|>user<|end_header_id|> Original question: What treatment or management plan is proposed for a patient's closed head injury and multiple contusions?<|eot_id|>assistant", '', 'Here are three alternative versions of the original question:', '', 'What are the recommended medical interventions for a patient with a closed head injury and multiple contusions?', '', 'Can you provide information on the standard treatment protocols for patients who 


    <|start_header_id|>system<|end_header_id|> Answer as an expert doctor. Analyse the context prvided below to answer the query. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context:
    [Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '786.05', 'chief_complaint': 'PLEURAL EFFUSION', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '428.0,427.31,242.90,V10.3,250.00,401.9,', 'downlaod_time': '', 'source': 'report36.xml', 'start_index': 0, 'subtype': 'CONSULT', 'type': 'HP', 'year': '2007'}), Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '733.82', 'chief_complaint': 'PINS STICKING THROUGH FEMUR', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '733.82,996.49,V45.4,305.1,305.20,', 'downlaod_time': '', 'source': 'report20.xml', 'start_index': 0, 'subtype': 'GIM CONSULT', 'type': 'HP', 'year

In [41]:
answer = rag_chain.invoke("What is the physical examination of Mark Johnson")
print(answer['text'])
len(answer['context'])

INFO:langchain.retrievers.multi_query:Generated queries: ['<|start_header_id|>system<|end_header_id|> ', '    You are an AI language model assistant. Your task is to generate three (3) different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.', '    Provide these alternative questions separated by newlines.', '    <|eot_id|>', '    <|start_header_id|>user<|end_header_id|> Original question: What is the physical examination of Mark Johnson<|eot_id|>assistant', '', 'Here are three alternative versions of the original question:', '', "What are the key findings in Mark Johnson's physical examination report?", '---------------------------------------------------------', '', '*Note: This version focuses on the specific details mentioned in the report.*', '', "What can be learned about Mark 


    <|start_header_id|>system<|end_header_id|> Answer as an expert doctor. Analyse the context prvided below to answer the query. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context:
    [Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '786.05', 'chief_complaint': 'PLEURAL EFFUSION', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '428.0,427.31,242.90,V10.3,250.00,401.9,', 'downlaod_time': '', 'source': 'report36.xml', 'start_index': 0, 'subtype': 'CONSULT', 'type': 'HP', 'year': '2007'}), Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '733.82', 'chief_complaint': 'PINS STICKING THROUGH FEMUR', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '733.82,996.49,V45.4,305.1,305.20,', 'downlaod_time': '', 'source': 'report20.xml', 'start_index': 0, 'subtype': 'GIM CONSULT', 'type': 'HP', 'year

30

In [44]:
answer = rag_chain.invoke("The patient has burning in the tongue, what may be the cause?")
print(answer['text'])
len(answer['context'])

INFO:langchain.retrievers.multi_query:Generated queries: ['<|start_header_id|>system<|end_header_id|> ', '    You are an AI language model assistant. Your task is to generate three (3) different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.', '    Provide these alternative questions separated by newlines.', '    <|eot_id|>', '    <|start_header_id|>user<|end_header_id|> Original question: The patient has burning in the tongue, what may be the cause?<|eot_id|>assistant', '', 'Here are three alternative versions of the original question:', '', '1. What medical conditions or diseases are commonly associated with a sensation of burning in the tongue?', '', '2. Can you provide information on potential causes of a burning sensation in the mouth, specifically related to the tongue?', '', '


    <|start_header_id|>system<|end_header_id|> Answer as an expert doctor. Analyse the context prvided below to answer the query. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context:
    [Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '786.05', 'chief_complaint': 'PLEURAL EFFUSION', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '428.0,427.31,242.90,V10.3,250.00,401.9,', 'downlaod_time': '', 'source': 'report36.xml', 'start_index': 0, 'subtype': 'CONSULT', 'type': 'HP', 'year': '2007'}), Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '733.82', 'chief_complaint': 'PINS STICKING THROUGH FEMUR', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '733.82,996.49,V45.4,305.1,305.20,', 'downlaod_time': '', 'source': 'report20.xml', 'start_index': 0, 'subtype': 'GIM CONSULT', 'type': 'HP', 'year

22

In [27]:
# report 18
answer = rag_chain.invoke("What are the recommended medical interventions for a patient experiencing severe lower abdominal pain?")
print(answer['text'])
len(answer['context'])

INFO:langchain.retrievers.multi_query:Generated queries: ['<|start_header_id|>system<|end_header_id|> ', '    You are an AI language model assistant. Your task is to generate three (3) different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.', '    Provide these alternative questions separated by newlines.', '    <|eot_id|>', '    <|start_header_id|>user<|end_header_id|> Original question: What are the recommended medical interventions for a patient experiencing severe lower abdominal pain?<|eot_id|>assistant', '', 'Here are three alternative versions of the original question:', '', 'What are the most effective medical treatments for patients presenting with acute lower abdominal pain?', '', 'Can you provide information on the best practices for managing severe lower abdominal pain i


    <|start_header_id|>system<|end_header_id|> Answer as an expert doctor. Analyse the context prvided below to answer the query. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context:
    [Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '786.50', 'chief_complaint': 'CHEST PAIN', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '786.50,401.9,493.90,272.4,V58.66,', 'downlaod_time': '', 'source': 'report23.xml', 'start_index': 0, 'subtype': '', 'type': 'HP', 'year': '2007'}), Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '562.10', 'chief_complaint': 'DIVERTICULITIS', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '584.9,518.0,403.91,428.0,486,276.2,562.11,585.4,518.3,244.9,414.01,250.80,791.0,250.50,362.01,285.29,', 'downlaod_time': '', 'source': 'report10.xml', 'start_index': 0, 'subtype'

29

In [27]:
answer = rag_chain.invoke("What treatment or management plan is proposed for a patient with an incision and drainage of the left ring finger?")
print(answer['text'])
len(answer['context'])

INFO:langchain.retrievers.multi_query:Generated queries: ['<|start_header_id|>system<|end_header_id|> ', '    You are an AI language model assistant. Your task is to generate three (3) different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.', '    Provide these alternative questions separated by newlines.', '    <|eot_id|>', '    <|start_header_id|>user<|end_header_id|> Original question: What treatment or management plan is proposed for a patient with an incision and drainage of the left ring finger?<|eot_id|>assistant', '', 'Here are three alternative versions of the original question:', '', 'What are the typical post-operative care instructions for a patient who has undergone incision and drainage of the left ring finger?', '', 'What is the recommended wound care protocol for a p


    <|start_header_id|>system<|end_header_id|> Answer as an expert doctor. Analyse the context prvided below to answer the query. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context:
    [Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '789.00', 'chief_complaint': 'PANCREATITIS', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '577.0,428.0,496,250.02,599.0,244.9,272.4,729.1,414.00,V45.81,530.81,278.01,E933.1,714.0,715.89,721.3,327.23,', 'downlaod_time': '', 'source': 'report40.xml', 'start_index': 0, 'subtype': 'CONSULT', 'type': 'HP', 'year': '2007'}), Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '786.50', 'chief_complaint': 'CHEST PAIN', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '786.50,401.9,493.90,272.4,V58.66,', 'downlaod_time': '', 'source': 'report23.xml', 'start_index': 

24

In [28]:
answer = rag_chain.invoke("How to treat an infection of the patella at a bone")
print(answer['text'])
len(answer['context'])

INFO:langchain.retrievers.multi_query:Generated queries: ['<|start_header_id|>system<|end_header_id|> ', '    You are an AI language model assistant. Your task is to generate three (3) different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.', '    Provide these alternative questions separated by newlines.', '    <|eot_id|>', '    <|start_header_id|>user<|end_header_id|> Original question: How to treat an infection of the patella at a bone<|eot_id|>    system ', '    You are an AI language model assistant. Your task is to generate three (3) different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based sim


    <|start_header_id|>system<|end_header_id|> Answer as an expert doctor. Analyse the context prvided below to answer the query. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context:
    [Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '789.00', 'chief_complaint': 'PANCREATITIS', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '577.0,428.0,496,250.02,599.0,244.9,272.4,729.1,414.00,V45.81,530.81,278.01,E933.1,714.0,715.89,721.3,327.23,', 'downlaod_time': '', 'source': 'report40.xml', 'start_index': 0, 'subtype': 'CONSULT', 'type': 'HP', 'year': '2007'}), Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '786.50', 'chief_complaint': 'CHEST PAIN', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '786.50,401.9,493.90,272.4,V58.66,', 'downlaod_time': '', 'source': 'report23.xml', 'start_index': 

26

In [37]:
answer = rag_chain.invoke("What treatment or management plan is proposed for excessive tearing of the left eye")
print(answer['text'])
len(answer['context'])

INFO:langchain.retrievers.multi_query:Generated queries: ['<|start_header_id|>system<|end_header_id|> ', '    You are an AI language model assistant. Your task is to generate three (3) different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.', '    Provide these alternative questions separated by newlines.', '    <|eot_id|>', '    <|start_header_id|>user<|end_header_id|> Original question: What treatment or management plan is proposed for excessive tearing of the left eye<|eot_id|>assistant', '', 'Here are three alternative versions of the original question:', '', 'What are the recommended treatments and management strategies for epiphora (excessive tearing) affecting the left eye?', '', 'What is the typical course of action for managing chronic tearing in the left eye, including any


    <|start_header_id|>system<|end_header_id|> Answer as an expert doctor. Analyse the context prvided below to answer the query. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context:
    [Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '789.00', 'chief_complaint': 'PANCREATITIS', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '577.0,428.0,496,250.02,599.0,244.9,272.4,729.1,414.00,V45.81,530.81,278.01,E933.1,714.0,715.89,721.3,327.23,', 'downlaod_time': '', 'source': 'report40.xml', 'start_index': 0, 'subtype': 'CONSULT', 'type': 'HP', 'year': '2007'}), Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '786.50', 'chief_complaint': 'CHEST PAIN', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '786.50,401.9,493.90,272.4,V58.66,', 'downlaod_time': '', 'source': 'report23.xml', 'start_index': 

22

In [26]:
answer = rag_chain.invoke("Hello Doctor, my name is Andrew ying and my head hurts. Do you know why?")
print(answer['text'])
len(answer['context'])

INFO:langchain.retrievers.multi_query:Generated queries: ['<|start_header_id|>system<|end_header_id|> ', '    You are an AI language model assistant. Your task is to generate three (3) different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.', '    Provide these alternative questions separated by newlines.', '    <|eot_id|>', '    <|start_header_id|>user<|end_header_id|> Original question: Hello Doctor, my name is Andrew ying and my head hurts. Do you know why?<|eot_id|>system ', '    You are an AI language model assistant. Your task is to generate three (3) different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the 


    <|start_header_id|>system<|end_header_id|> Answer as an expert doctor. Analyse the context prvided below to answer the query. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context:
    [Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '789.00', 'chief_complaint': 'PANCREATITIS', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '577.0,428.0,496,250.02,599.0,244.9,272.4,729.1,414.00,V45.81,530.81,278.01,E933.1,714.0,715.89,721.3,327.23,', 'downlaod_time': '', 'source': 'report40.xml', 'start_index': 0, 'subtype': 'CONSULT', 'type': 'HP', 'year': '2007'}), Document(page_content='[Report de-identified (Safe-harbor compliant) by De-ID v.6.22.07.0]', metadata={'admit_diagnosis': '786.50', 'chief_complaint': 'CHEST PAIN', 'deid': 'v.6.22.07.0', 'discharge_diagnosis': '786.50,401.9,493.90,272.4,V58.66,', 'downlaod_time': '', 'source': 'report23.xml', 'start_index': 

27